# 클라이언트 API호출

In [1]:
import pandas as pd
import numpy as np
import requests
import json

### 1. 단일 값 호출

In [2]:
import requests

input = {'Pclass': 1,
 'Sex': 'male',
 'Age': 22.0,
 'SibSp': 0,
 'Parch': 0,
 'Fare':  71.89,
 'Embarked': 'Q'}
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() # 결과를 json 형태로 받아와 저장하지만, python에서는 딕셔너리로 저장됨.
result['pred']

['Died']

## 2. 여러 값 호출

In [3]:
input = {'Pclass': [1,2],
 'Sex': ['male','female'],
 'Age': [22.0, 34],
 'SibSp': [0, 2],
 'Parch': [0, 2],
 'Fare':  [71.89, 31],
 'Embarked': ['Q','S']}
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() # 결과를 json 형태로 받아와 저장하지만, python에서는 딕셔너리로 저장됨.
result['pred']

['Died', 'Survived']

## 3. 데이터셋(배치)으로 전달하기

* 데이터프레임으로 부터 정보 전달하기1

In [4]:
use_cols = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp' ,'Parch', 'Fare', 'Embarked']
data = pd.read_csv('data/new_data.csv', usecols = use_cols)
target = 'Survived'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [5]:
temp = x.sample(3)

In [6]:
input = temp.to_json(orient = 'columns')
input = json.loads(input)
input

{'Pclass': {'11': 3, '1': 1, '8': 1},
 'Sex': {'11': 'female', '1': 'male', '8': 'female'},
 'Age': {'11': 22.0, '1': 33.0, '8': 56.0},
 'SibSp': {'11': 0, '1': 0, '8': 0},
 'Parch': {'11': 0, '1': 0, '8': 1},
 'Fare': {'11': 10.5167, '1': 5.0, '8': 83.1583},
 'Embarked': {'11': 'S', '1': 'S', '8': 'C'}}

In [7]:
input = { key : list(temp[key]) for key in list(temp) }
input

{'Pclass': [3, 1, 1],
 'Sex': ['female', 'male', 'female'],
 'Age': [22.0, 33.0, 56.0],
 'SibSp': [0, 0, 0],
 'Parch': [0, 0, 1],
 'Fare': [10.5167, 5.0, 83.1583],
 'Embarked': ['S', 'S', 'C']}

In [8]:
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() 
result['pred']

['Survived', 'Died', 'Survived']

* 데이터프레임으로 부터 정보전달하고, 예측결과 합치기

In [9]:
# request
input = { key : list(x[key]) for key in list(x) }
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() 

In [10]:
# 결과 합치기
x['y'] = y
x['y_pred'] = result['pred']
x

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,y,y_pred
0,1,female,47.0,1,1,52.5542,S,1,Survived
1,1,male,33.0,0,0,5.0000,S,0,Died
2,3,male,47.0,0,0,9.0000,S,0,Died
3,2,female,28.0,1,0,24.0000,C,1,Survived
4,3,female,15.0,0,0,7.2250,C,1,Survived
5,3,male,20.0,0,0,9.8458,S,0,Died
6,3,male,19.0,0,0,7.8958,S,0,Died
7,3,male,NaN,0,0,7.8958,S,0,Died
8,1,female,56.0,0,1,83.1583,C,1,Survived
9,2,female,25.0,0,1,26.0000,S,1,Survived
